In [ ]:
# Import dependencies

import os
import csv
import pandas as pd

# Define both files to be analyzed

file_1 = "final_clean_dog_data.csv"
file_2 = "final_wheelchair_data.csv"

# Read both files

dog_df = pd.read_csv(file_1, encoding="UTF-8", low_memory = False)    
wc_df = pd.read_csv(file_2, encoding="UTF-8", low_memory = False)

# Preview dog restaurant data frame

dog_df.head()

In [ ]:
# Preview wheel chair restaurant data frame

wc_df.head()

In [ ]:
# Make sure rows of data are correct

print(len(dog_df))
print(len(wc_df))

In [ ]:
# Combine both data frames so that restaurant data can be analyzed

combined_df = pd.concat([dog_df, wc_df])

# Check number of rows of data frame

len(combined_df)

In [ ]:
# Get all rows that have duplicate restaurant names

duplicate_rows = combined_df[combined_df.duplicated("name", keep=False)]

# See how many restaurants are duplicated by name

len(duplicate_rows)

In [ ]:
# Alphabetize the restaurant names so that the entries could be compared
# Some restaurants are chains so need to look at more than just duplicated names

duplicate_rows = duplicate_rows.sort_values("name", ascending=True)

# Preview the data frame

duplicate_rows.head()

In [ ]:
# Also, the latitude/longitude recorded for each restaurant will not be exactly the same so cannot compare by lat and long
# Different scraped sites have different entires for the same restaurant
# To find if the restaurant is the same, find the difference in the latitude from the previous row
# Since the restaurants are in alphabetical order, if the restaurant is a duplicate, the latitude should be relatively close to the previous entry
# Create a new column "lat_diff" to hold the difference values

duplicate_rows["lat_diff"] = duplicate_rows['latitude'] - duplicate_rows['latitude'].shift(1)

# Preview data frame

duplicate_rows.head()

In [ ]:
# Since the latitude can also be off by a little bit, will analyze another column as well
# If the restaurant has the same name within the same zip code with a similar latitude, the restaurant should be a duplicate
# The difference in the zip code should be zero for duplicates
# Create new column that analyzes the difference in zip code from previous entry

duplicate_rows["zip_diff"] = duplicate_rows['zip_code'] - duplicate_rows['zip_code'].shift(1)

# Display data frame

duplicate_rows.head()

In [ ]:
# See which restaurants are duplicates
# The data that results should be the restaurants that are both wheel chair accessible and dog friendly
# The duplicate entires means that the restaurant shows up in both the wheel chair and dog friendly data sets
# Must find the absolute value of the latitude difference as some may be negative
# A latitude difference less than 0.0015 was chosen to compare duplicate entries
# The zip code difference should be equal to zero as well for the duplicates

both_wc_and_dog = duplicate_rows[(abs(duplicate_rows["lat_diff"]) < 0.0015) & (duplicate_rows["zip_diff"] == 0)]

# Display the data frame

len(both_wc_and_dog)

In [ ]:
# Analyze the data collected

both_df = both_wc_and_dog.sort_values("name", ascending=True)

In [ ]:
# Drop any duplicate entries
# Find by restaurant name since the data is already filtered by zip and latitude difference

clean_df = both_wc_and_dog.drop_duplicates("name", keep="first")

# Preview data frame

len(clean_df)

In [ ]:
# Get a clean data frame with only the columns needed

clean_df = clean_df[["name", "latitude", "longitude", "address", "zip_code", "website"]]

In [ ]:
# Save the data frame to a CSV file

clean_df.to_csv("final_data_both_dog_and_wheelchair.csv", index=False)